# 自然语言处理之长文本分类

## 实验背景

目前人工智能的发展在运算智能和感知智能已经取得了很大的突破和优于人类的表现。而在以理解人类语言为入口的认知智能上，目前现代自然语言处理技术已经可以实现文档自动解析、关键信息提取、文本分类审核、文本智能纠错等一定基础性的文字处理工作，并在各行各业得到充分应用。

自然语言处理一直是人工智能领域的重要话题，而人类语言的复杂性也给 NLP 布下了重重困难等待解决。长文本的智能解析就是颇具挑战性的任务，如何从纷繁多变、信息量庞杂的冗长文本中获取关键信息，一直是文本领域难题。随着深度学习的热潮来临，有许多新方法来到了 NLP 领域，给相关任务带来了更多优秀成果，也给大家带来了更多应用和想象的空间。

## 实验目的

建立模型通过长文本数据正文(article)，预测文本对应的类别(class)。

## 实验内容

<a href='#数据介绍'>数据介绍</a><br/>
<a href='#实验步骤'>实验步骤</a><br/>
<a href='#结果文件说明'>结果文件说明</a><br/>

### 数据介绍

数据集分为训练集、测试集两个部分。

#### 训练集

train_set.csv：此数据集用于训练模型，每一行对应一篇文章。文章分别在“字”和“词”的级别上做了脱敏处理。共有四列：

1)第一列是文章的索引(id)；

2)第二列是文章正文在“字”级别上的表示，即字符相隔正文(article)；

3)第三列是在“词”级别上的表示，即词语相隔正文(word_seg)；

4)第四列是这篇文章的标注(class)。

注：每一个数字对应一个“字”，或“词”，或“标点符号”。“字”的编号与“词”的编号是独立的。

#### 测试集

test_set.csv：此数据用于测试。数据格式同train_set.csv，但不包含class。

注：test_set与train_test中文章id的编号是独立的。

### 实验步骤

#### 1.创建tf、tfidf、doc2vec特征

```python
#用途：将原始数据数字化为tf特征，并将结果保存至本地

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import time

t_start = time.time()

#1.数据预处理
# 根据设置后面参数，nrows的值，来设置取数据集中数据量的多少（默认为全部取出）

df_train = pd.read_csv('../Data/train_set.csv',nrows=200)
df_train.drop(columns='article', inplace=True)
df_test = pd.read_csv('../Data/test_set.csv',nrows=200)
df_test.drop(columns='article', inplace=True)
df_all = pd.concat(objs=[df_train, df_test], axis=0, sort=True)
y_train = (df_train['class'] - 1).values

#2.特征工程

vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=100, max_df=0.8)
vectorizer.fit(df_all['word_seg'])
x_train = vectorizer.transform(df_train['word_seg'])
x_test = vectorizer.transform(df_test['word_seg'])

#3.保存至本地

data = (x_train, y_train, x_test)
fp = open('../data_tf.pkl', 'wb')
pickle.dump(data, fp)
fp.close()

t_end = time.time()
print("已将原始数据数字化为tf特征，共耗时：{}min".format((t_end-t_start)/60))
```

```python
#用途: 将原始数据数字化为tfidf特征，并将结果保存至本地

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import time

t_start = time.time()

#1.数据预处理

df_train = pd.read_csv('../Data/train_set.csv',nrows=200)
df_train.drop(columns='article', inplace=True)
df_test = pd.read_csv('../Data/test_set.csv',nrows=200)
df_test.drop(columns='article', inplace=True)
f_all = pd.concat(objs=[df_train, df_test], axis=0, sort=True)
y_train = (df_train['class'] - 1).values

#2.特征工程

vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=3, max_df=0.9, sublinear_tf=True)
vectorizer.fit(df_train['word_seg'])
x_train = vectorizer.transform(df_train['word_seg'])
x_test = vectorizer.transform(df_test['word_seg'])

#3.保存至本地

data = (x_train, y_train, x_test)
fp = open('../data_tfidf.pkl', 'wb')
pickle.dump(data, fp)
fp.close()

t_end = time.time()
print("已将原始数据数字化为tfidf特征，共耗时：{}min".format((t_end-t_start)/60))
```

Doc2vec简介

Doc2Vec 或者叫做 paragraph2vec, sentence embeddings，是一种非监督式算法，可以获得 sentences/paragraphs/documents 的向量表达，是 word2vec 的拓展。学出来的向量可以通过计算距离来找 sentences/paragraphs/documents 之间的相似性，可以用于文本聚类，对于有标签的数据，还可以用监督学习的方法进行文本分类，例如经典的情感分析问题。

Doc2vec参数

```
1)documents【默认None】
语料，可以是一个list，对于大语料集，建议使用BrownCorpus、Text8Corpus或lineSentence构建。

2)vector_size【默认100】
特征向量的维度。

3)alpha【默认0.025】
初始的学习速率，在训练过程中会线性地递减到min_alpha（学习率的最小值）

4)window【默认5】
窗口大小，表示当前词与预测词在一个句子中的最大距离是多少。

5)min_count【默认5】
可以对字典做截断. 词频少于min_count次数的单词会被丢弃掉。

6)sample【默认1e-3】
高频词汇的随机降采样的配置阈值，范围是(0,1e-5)。

7)workers【默认3】
用于控制训练的并行数。

8)dm【默认1】
训练算法：默认为1，指DM；dm=0，则使用DBOW。

9)epochs
迭代次数。

10)dm_mean【默认0】
当使用DM训练算法时，对上下文向量相加（默认0）；若设为1，则求均值。

11)dm_concat【默认0】
当设为1时，在使用DM训练算法时，直接将上下文向量和Doc向量拼接。

12)dbow_words【默认0】
当设为1时，则在训练doc_vector（DBOW）的同时训练Word_vector（Skip-gram）；默认为0，只训练doc_vector，速度更快。
```
在本课题中，我们直接调用Doc2Vec模型即可，具体代码如下所示：

```python
#用途：将原始数据数字化为doc2vec特征，并将结果保存至本地

import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import time
import pickle

t_start = time.time()

#1.辅助函数

def sentence2list(sentence):
    s_list = sentence.strip().split()
    return s_list

#2.读取原始数据，并进行简单处理

df_train = pd.read_csv('../Data/train_set.csv',nrows=200)
df_train.drop(columns='article', inplace=True)
df_test = pd.read_csv('../Data/test_set.csv',nrows=200)
df_test.drop(columns='article', inplace=True)
df_all = pd.concat(objs=[df_train, df_test], axis=0, sort=True)
y_train = (df_train['class'] - 1).values

df_all['word_list'] = df_all['word_seg'].apply(sentence2list)
texts = df_all['word_list'].tolist()

#3.doc2vec

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]
model = Doc2Vec(documents, vector_size=200, window=5, min_count=3, workers=4, epochs=25)
docvecs = model.docvecs

x_train = []
for i in range(0, 200):
    x_train.append(docvecs[i])
x_train = np.array(x_train)

x_test = []
for j in range(200, 400):
    x_test.append(docvecs[j])
x_test = np.array(x_test)

#4.将doc2vec特征保存至本地

data = (x_train, y_train, x_test)
f_data = open('../data_doc2vec_25.pkl', 'wb')
pickle.dump(data, f_data)
f_data.close()

t_end = time.time()
print("已将原始数据数字化为doc2vec特征，共耗时：{}min".format((t_end-t_start)/60))
```

#### 2.对1中创建的特征进行特征选择

```python
#进行特征选择：对特征进行嵌入式选择

import time
import pickle
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

t_start = time.time()

#1.读取特征

#此处需要改为带选择的特征地址（上述1中共生成3个pkl文件）
features_path = '../data_doc2vec_25.pkl'
fp = open(features_path, 'rb')
x_train, y_train, x_test = pickle.load(fp)
fp.close()

#2.进行特征选择

alo_name = 'LSVC_l2'
lsvc = LinearSVC(penalty='l2', C=1.0, dual=True, max_iter=50000).fit(x_train, y_train)
slt = SelectFromModel(lsvc, prefit=True)
x_train_s = slt.transform(x_train)
x_test_s = slt.transform(x_test)

#3.保存选择后的特征至本地

num_features = x_train_s.shape[1]
data_path = '../' + features_path.split('.')[-2] + '_select_' + alo_name + '_' + str(num_features) + '.pkl'
data_f = open(data_path, 'wb') 
pickle.dump((x_train_s, y_train, x_test_s), data_f)
data_f.close()

t_end = time.time()
print("特征选择完成，选择{}个特征，共耗时{}min".format(num_features, (t_end-t_start)/60))

```

#### 3.创建LDA、LSA特征

LDA特征

LDA（Latent Dirichlet Allocation）是一种文档主题生成模型，也称为一个三层贝叶斯概率模型，包含词、主题和文档三层结构。所谓生成模型，就是说我们认为一篇文章的每个词都是通过“以一定概率选择了某个主题，并从这个主题中以一定概率选择某个词语”这样一个过程得到。文档到主题服从多项式分布，主题到词服从多项式分布。LDA是一种非监督机器学习技术，可以用来识别大规模文档集（document collection）或语料库（corpus）中潜藏的主题信息。它采用了词袋（bag of words）的方法，这种方法将每一篇文档视为一个词频向量，从而将文本信息转化为了易于建模的数字信息。LDA模型学习时的训练数据并不是一篇篇文本，而是Document-word matrix，它可以是array也可以是稀疏矩阵，维数是n_samples*n_features，其中n_features为词(term)的个数。因此在训练LDA主题模型前，需要先利用CountVectorizer统计词频并保存。

```
LatentDirichletAllocation类的主要输入参数：

1)n_topics【需要调参】
即我们的隐含主题数KK。KK的大小取决于我们对主题划分的需求：如果需要区分的类别很少，则KK的取值可去较小；若目标类别较多，则KK值需要取很大，并且需要的训练文档数量很多。

2)doc_topic_prior【默认1/K1/K】
即我们的文档主题先验Dirichlet分布θdθd的参数αα。一般如果我们没有主题分布的先验知识就使用默认值。

3)topic_word_prior【默认1/K1/K】
即我们的主题词先验Dirichlet分布βkβk的参数ηη。一般如果我们没有主题分布的先验知识就可以使用默认值。

4)learning_method【默认online】
即LDA的求解算法。有 ‘batch’ 和 ‘online’两种选择。 ‘batch’即我们在原理篇讲的变分推断EM算法，而“online”即在线变分推断EM算法，在“batch”的基础上引入了分步训练，将训练样本分批，逐步一批批的用样本更新主题词分布的算法。

5)learning_decay
仅仅在算法使用“online”时有意义，取值最好在(0.5, 1.0]，以保证“online”算法渐进的收敛。主要控制“online”算法的学习率，默认是0.7。一般不用修改这个参数。

6)learning_offset
仅仅在算法使用“online”时有意义，取值要大于1。用来减小前面训练样本批次对最终模型的影响。

7)max_iter【默认1000】
EM算法的最大迭代次数。

8)total_samples
仅仅在算法使用“online”时有意义， 即分步训练时每一批文档样本的数量。在使用partial_fit函数时需要。

9)batch_size
仅仅在算法使用“online”时有意义， 即每次EM算法迭代时使用的文档样本的数量。

10)mean_change_tol
即E步更新变分参数的阈值，所有变分参数更新小于阈值则E步结束，转入M步。

11)max_doc_update_iter
即E步更新变分参数的最大迭代次数，如果E步迭代次数达到阈值，则转入M步。
```
在本课题中，读取TF特征后直接调用LatentDirichletAllocation，然后标准化训练集和测试集即可，具体代码如下：

```python
#用途：将tf特征降维为lda特征，并将结果保存至本地

from sklearn.decomposition import LatentDirichletAllocation
import pickle
import time

t_start = time.time()

#1.tf特征加载

tf_path = '../data_tf_select_LSVC_l2_56.pkl'
f_tf = open(tf_path, 'rb')
x_train, y_train, x_test = pickle.load(f_tf)
f_tf.close()

#2.特征降维：lda

print("lda......")
lda = LatentDirichletAllocation(n_components=200)
x_train = lda.fit_transform(x_train)
x_test = lda.transform(x_test)

#3.将lda特征保存至本地

data = (x_train, y_train, x_test)
f_data = open('../data_lda.pkl', 'wb')
pickle.dump(data, f_data)
f_data.close()

t_end = time.time()
print("lda特征完成，共耗时：{}min".format((t_end-t_start)/60))
```

LSA简介

LSA （浅层语义分析）是一种自然语言处理中用到的方法，其通过“矢量语义空间”来提取文档与词中的“概念”，进而分析文档与词之间的关系。

LSA的基本假设是，如果两个词多次出现在同一文档中，则这两个词在语义上具有相似性。LSA使用大量的文本上构建一个矩阵，这个矩阵的一行代表一个词，一列代表一个文档，矩阵元素代表该词在该文档中出现的次数，然后再此矩阵上使用奇异值分解（SVD）来保留列信息的情况下减少矩阵行数，之后每两个词语的相似性则可以通过其行向量的cos值（或者归一化之后使用向量点乘）来进行标示，此值越接近于1则说明两个词语越相似，越接近于0则说明越不相似。

LSA降维

LSA 使用词-文档矩阵来描述一个词语是否在一篇文档中。词-文档矩阵式一个稀疏矩阵，其行代表词语，其列代表文档。一般情况下，词-文档矩阵的元素是该词在文档中的出现次数，也可以是是该词语的tf-idf(term frequency–inverse document frequency)。词-文档矩阵和传统的语义模型相比并没有实质上的区别，只是因为传统的语义模型并不是使用“矩阵”这种数学语言来进行描述。

在构建好词-文档矩阵之后，LSA将对该矩阵进行降维，来找到词-文档矩阵的一个低阶近似。降维的原因有以下几点：

1)原始的词-文档矩阵太大导致计算机无法处理，从此角度来看，降维后的新矩阵式原有矩阵的一个近似。

2)原始的词-文档矩阵中有噪音，从此角度来看，降维后的新矩阵式原矩阵的一个去噪矩阵。

3)原始的词-文档矩阵过于稀疏。原始的词-文档矩阵精确的反映了每个词是否“出现”于某篇文档的情况，然而我们往往对某篇文档“相关”的所有词更感兴趣，因此我们需要发掘一个词的各种同义词的情况。

降维后的语义空间可以用于以下方面：

1)在低维语义空间可对文档进行比较，进而可用于文档聚类和文档分类。

2)在翻译好的文档上进行训练，可以发现不同语言的相似文档，可用于跨语言检索。

3)发现词与词之间的关系，可用于同义词、歧义词检测。.

4)通过查询映射到语义空间，可进行信息检索。

5)从语义的角度发现词语的相关性，可用于“选择题回答模型”（multi choice qustions answering model）。

在本课题中，我们读取tfidf特征后直接调用TruncatedSVD ()即可，具体代码如下所示：

```python
#用途：将tfidf特征降维为lsa特征，并将结果保存至本地

from sklearn.decomposition import TruncatedSVD
import pickle
import time

t_start = time.time()

#1.读取tfidf特征

tfidf_path = '../data_tfidf_select_LSVC_l2_2992.pkl'
f_tfidf = open(tfidf_path, 'rb')
x_train, y_train, x_test = pickle.load(f_tfidf)
f_tfidf.close()

#2.特征降维：lsa

print("lsa......")
lsa = TruncatedSVD(n_components=200)
x_train = lsa.fit_transform(x_train)
x_test = lsa.transform(x_test)

#3.将lsa特征保存至本地

data = (x_train, y_train, x_test)
f_data = open('../data_s_lsvc_l2_143w_lsa.pkl', 'wb')
pickle.dump(data, f_data)
f_data.close()

t_end = time.time()
print("lsa特征完成，共耗时：{}min".format((t_end-t_start)/60))

```

#### 4.将lda/lsa/doc2vec三种特征进行特征融合

```python
#融合特征：lda/lsa/doc2vec三种特征进行特征融合，并将结果保存至本地

import numpy as np
import pickle
import time

t_start = time.time()

#1.读取lda/lsa/doc2vec特征，并对这三种特征进行拼接融合

f1 = open('../data_lda.pkl', 'rb')
x_train_1, y_train, x_test_1 = pickle.load(f1)
f1.close()

f2 = open('../data_s_lsvc_l2_143w_lsa.pkl', 'rb')
x_train_2, y_train, x_test_2 = pickle.load(f2)
f2.close()

f3 = open('../data_doc2vec_25.pkl', 'rb')
x_train_3, _, x_test_3 = pickle.load(f3)
f3.close()

x_train = np.concatenate((x_train_1, x_train_2, x_train_3), axis=1)
x_test = np.concatenate((x_test_1, x_test_2, x_test_3), axis=1)

#2.将融合后的特征，保存至本地

data = (x_train, y_train, x_test)
fp = open('../data_ensemble.pkl', 'wb')
pickle.dump(data, fp)
fp.close()

t_end = time.time()
print("已将原始数据数字化为融合的特征，共耗时：{}min".format((t_end-t_start)/60))

```

#### 5.将特征转换为稀疏矩阵

```python
#简介：将data_ensemble特征转换为稀疏矩阵

import pickle
from scipy import sparse
from scipy.sparse import hstack

#1.读取ensemble特征

f_ensemble = open('../data_ensemble.pkl', 'rb')
x_train_ens, y_train, x_test_ens = pickle.load(f_ensemble)
f_ensemble.close()

#2.将numpy数组转换为csr稀疏矩阵

x_train_ens_s = sparse.csr_matrix(x_train_ens)
x_test_ens_s = sparse.csc_matrix(x_test_ens)

#3.读取tfidf特征

f_tfidf = open('../data_tfidf_select_LSVC_l2_2992.pkl', 'rb')
x_train_tfidf, _, x_test_tfidf = pickle.load(f_tfidf)
f_tfidf.close()

#4.对两个稀疏矩阵进行合并

x_train_spar = hstack([x_train_ens_s, x_train_tfidf])
x_test_spar = hstack([x_test_ens_s, x_test_tfidf])

#5.将合并后的稀疏特征保存至本地

data = (x_train_spar, y_train, x_test_spar)
f = open('../data_ensemble_spar.pkl', 'wb')
pickle.dump(data, f)
f.close()
```

#### 6.选择使用特征和使用模型，训练模型并生成结果

```python
#根据features_path中的数据，对机器学习模型进行训练，对测试集进行预测，并将结果保存至本地

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import time
import pickle

t_start = time.time()

#是否开启验证集模式
status_vali = True

#特征存储的路径
features_path = '../data_ensemble_spar.pkl'

#修改clf_name可对学习算法进行选择；修改base_clf改变集成学习的基分类器
clf_name = 'xgb'

base_clf = LinearSVC()

clfs = {
    'lr': LogisticRegression(penalty='l2', C=1.0),
    'svm': LinearSVC(penalty='l2', dual=True),
    'bagging': BaggingClassifier(base_estimator=base_clf, n_estimators=60, max_samples=1.0, max_features=1.0, random_state=1,
                        n_jobs=1, verbose=1),
    'rf': RandomForestClassifier(n_estimators=10, criterion='gini'),
    'adaboost': AdaBoostClassifier(base_estimator=base_clf, n_estimators=50),
    'gbdt': GradientBoostingClassifier(),
    'xgb': xgb.XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=100, silent=True, objective='multi:softmax',
                        nthread=1, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1,
                        colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, seed=0,
                        missing=None),
    'lgb': lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=250,
                              max_bin=255, subsample_for_bin=200000, objective=None, min_split_gain=0.0, min_child_weight=0.001,
                              min_child_samples=20, subsample=1.0, subsample_freq=1, colsample_bytree=1.0, reg_alpha=0.0,
                              reg_lambda=0.5, random_state=None, n_jobs=-1, silent=True)
}
clf = clfs[clf_name]

#读取数据
data_fp = open(features_path, 'rb')
x_train, y_train, x_test = pickle.load(data_fp)
data_fp.close()

#划分训练集和验证集，验证集比例为test_size
if status_vali:
    x_train, x_vali, y_train, y_vali = train_test_split(x_train, y_train,     test_size=0.1, random_state=0)
    
#训练分类器
clf.fit(x_train, y_train)

#在验证集上评估模型
if status_vali:
    pre_vali = clf.predict(x_vali)
    score_vali = f1_score(y_true=y_vali, y_pred=pre_vali, average='macro')
    print("验证集分数：{}".format(score_vali))

#对测试集进行预测;将预测结果转换为官方标准格式；并将结果保存至本地
y_test = clf.predict(x_test) + 1
df_result = pd.DataFrame(data={'id':range(200), 'class': y_test.tolist()})
result_path = '../Submission/' + features_path.split('/')[-1] + '_sklearn_' + clf_name + '.csv'
df_result.to_csv(result_path, index=False)

t_end = time.time()
print("训练结束，耗时:{}min".format((t_end - t_start) / 60))
```

#### 7.重复6步，选择不同的特征与模型组合。

### 结果文件说明

建立代码的同级文件夹三个：Data、Code、Submission


Data：用于存放课题需要的数据

Code：用于存放代码文件和课题使用特征、模型

Submission：用于存放预测结果